# Analysis of P21 & P18 from pilot data
B.L. de Vries - 3/12/2018

The goal of this notebook is to discuss the results and analysis I have done. This notebook will not run locally on you system

In [1]:
%load_ext autoreload
%autoreload 2

Loading packages

In [2]:
import os
import sys
import glob
import time
import pandas as pd

In [7]:
import explibrosa
import exploface
import elanwriter
print(explibrosa.__version__, 
exploface.__version__, 
elanwriter.__version__)

0.0.0.dev1 0.0.0.dev6 0.0.0.dev2


## Directories used
First I will make variables holding the different directories with input and output files and the executables of openface

I use openface via a python wrapper script ```analyse_video``` that I made. This is just a very simple wrapper. I also define the openface executable here.

In [8]:
openface_runner_path = os.path.join("C:\\","Users","bldev","Dropbox","0_DATA_BEN","WORK_ESCIENCE",
                                    "PROJECTS","DEMENTIA","TOOLS","openface_runner")
sys.path.append(openface_runner_path)
from process_video import analyse_video

openface_version_dir = "OpenFace_2.0.5_win_x64"
openface_exe = os.path.join("C:\\","Users","bldev","Dropbox","0_DATA_BEN","WORK_ESCIENCE",
                            "PROJECTS","DEMENTIA","TOOLS","openface_runner",
                            openface_version_dir,"FeatureExtraction.exe")

The input directories:

In [9]:
encrypted_dir = os.path.join("E:\\","PROJECT_DATA","DEMENTIA_2017")
video_dir = os.path.join(encrypted_dir,"Database")
audio_dir = os.path.join(encrypted_dir,"Database")#os.path.join(encrypted_dir,"Database","P21","S2","Audio")

The output directories:

In [17]:
output_privacy_dir = os.path.join(encrypted_dir, "Database_ANALYSIS", "P21_18_29JAN2019")

output_openface_dir = os.path.join(output_privacy_dir, "openface_output")
output_audio_dir = os.path.join(output_privacy_dir, "audio_output")
output_merged_dir = os.path.join(output_privacy_dir, "merged_output")


Check if input and output directories exist:

In [18]:
print("openface exe found: ",os.path.isfile(openface_exe))
print("video input dir: ", os.path.isdir(video_dir))
#print("audio input dir: ", os.path.isdir(audio_dir))
print("merged output dir: ", os.path.isdir(output_merged_dir))
print("video output dir: ", os.path.isdir(output_openface_dir))
print("audio output dir: ", os.path.isdir(output_audio_dir))
#output_openface_privacy_dir

openface exe found:  True
video input dir:  True
merged output dir:  True
video output dir:  True
audio output dir:  True


Here I collect all the video and audio files we need to process

In [12]:
audio_file_paths = glob.glob(os.path.join(video_dir, "**/*.wav"))
audio_file_names = list(map(os.path.basename, audio_file_paths))

In [13]:
video_file_paths = glob.glob(os.path.join(video_dir, "**/*.mp4"))
video_file_names = list(map(os.path.basename, video_file_paths))

In [14]:
print(video_file_paths)
file_names = list(zip(video_file_names, audio_file_names))
file_names

['E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P18\\P18_S2_Demograph_Cfront.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P18\\P18_S2_IAPS_HAPPY_Cfront.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P18\\P18_S2_IAPS_SAD_Cfront.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P18\\P18_S2_LSB_HM1_Cfront.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P18\\P18_S2_LSB_SM1_Cfront.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P18\\P18_S2_LSB_SM2_Cfront.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P18\\P18_S2_Neutral_2_Cfront.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P21\\P21_S2_IAPS_HAPPY_C1.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P21\\P21_S2_IAPS_SAD_C1.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P21\\P21_S2_LSB_HM1_C1.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P21\\P21_S2_LSB_HM2_C1.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P21\\P21_S2_LSB_HM3_C1.mp4', 'E:\\PROJECT_DATA\\DEMENTIA_2017\\Database\\P21\\P21_S2_LSB_SM1_C1.mp4', 'E:\\PROJE

[('P18_S2_Demograph_Cfront.mp4', 'P18_S2_Demograph.wav'),
 ('P18_S2_IAPS_HAPPY_Cfront.mp4', 'P18_S2_IAPS_HAPPY.wav'),
 ('P18_S2_IAPS_SAD_Cfront.mp4', 'P18_S2_IAPS_SAD.wav'),
 ('P18_S2_LSB_HM1_Cfront.mp4', 'P18_S2_LSB_HM1.wav'),
 ('P18_S2_LSB_SM1_Cfront.mp4', 'P18_S2_LSB_SM1.wav'),
 ('P18_S2_LSB_SM2_Cfront.mp4', 'P18_S2_LSB_SM2.wav'),
 ('P18_S2_Neutral_2_Cfront.mp4', 'P18_S2_Neutral_2.wav'),
 ('P21_S2_IAPS_HAPPY_C1.mp4', 'P21_S2_IAPS_HAPPY.wav'),
 ('P21_S2_IAPS_SAD_C1.mp4', 'P21_S2_IAPS_SAD.wav'),
 ('P21_S2_LSB_HM1_C1.mp4', 'P21_S2_LSB_HM1.wav'),
 ('P21_S2_LSB_HM2_C1.mp4', 'P21_S2_LSB_HM2.wav'),
 ('P21_S2_LSB_HM3_C1.mp4', 'P21_S2_LSB_HM3.wav'),
 ('P21_S2_LSB_SM1_C1.mp4', 'P21_S2_LSB_SM1.wav'),
 ('P21_S2_LSB_SM2_C1.mp4', 'P21_S2_LSB_SM2.wav'),
 ('P21_S2_LSB_SM3_C1.mp4', 'P21_S2_LSB_SM3.wav')]

## In this loop I run openface and librosa and store their time series.

In [15]:
print("Running videos from directory: ", video_dir)
for i, (v, a) in enumerate(file_names):
    print("- "+str(i+1)+"/"+str(len(file_names))+": "+v)
    
    audio_output_filename_no_ext = a.split(".")[0]
    audio_output_filename = audio_output_filename_no_ext+"_features.csv"
    
    # RUN AUDIO ANALYSIS
    print("   * Analyzing audio file: ", audio_file_paths[i])#os.path.join(audio_dir, a))
    if not os.path.isfile(os.path.join(output_audio_dir, audio_output_filename)):
        print("       Running, output generated in: ", output_audio_dir)
        start_time = time.time()
        time_series = explibrosa.get_feature_time_series(audio_file_paths[i],#os.path.join(audio_dir, a), 
                                                        output_audio_dir,
                                                        verbose=False)
    
        # META DATA LIBROSA
        with open(os.path.join(output_audio_dir, audio_output_filename_no_ext+".log"), "w") as text_file:
            print("Datetime: {}".format(time.strftime("%c")), file=text_file)
            for key, value in explibrosa.get_versions().items():
                print("Version {}: {}".format(key, value), file=text_file)
        print("       Execution time: %s min" % round((time.time() - start_time)/60, 2))
    else:
        print("      Output exists: "+os.path.join(output_audio_dir, audio_output_filename)[0:20]+\
              "..."+os.path.join(output_audio_dir, audio_output_filename)[-30:])
        
    # RUN OPENFACE
    openface_output_filename_no_ext = v.split(".")[0]
    openface_output_filename = openface_output_filename_no_ext+".csv"
    output_avi_openface = os.path.join(output_openface_dir, openface_output_filename_no_ext+".avi")
    print("   * Analyzing video: ", video_file_paths[i])#os.path.join(video_dir, v))
    if not os.path.isfile(os.path.join(output_openface_dir, openface_output_filename)):
        print("      Running, output generated in: ", output_openface_dir)
        start_time = time.time()
        analyse_video(verbose=False, 
                  inputpath = os.path.dirname(video_file_paths[i]),
                  filename = v,
                  outputpath = output_openface_dir, 
                  openface_feat_exe = openface_exe, 
                  execute=True)
        
        # META DATA OPENFACE AND EXPLOFACE
        with open(os.path.join(output_openface_dir, openface_output_filename_no_ext+".version_log"), "w") as text_file:
            print("Datetime: {}".format(time.strftime("%c")), file=text_file)
            print("Version exploface: {}".format(exploface.__version__), file=text_file)
            print("Version openface: {}".format(openface_version_dir), file=text_file)   
        print("       Execution time: %s min" % round((time.time() - start_time)/60, 2))
    else:
        print("      Output exists: "+os.path.join(output_openface_dir, openface_output_filename)[0:20]+\
              "..."+os.path.join(output_openface_dir, openface_output_filename)[-30:])



Running videos from directory:  E:\PROJECT_DATA\DEMENTIA_2017\Database
- 1/15: P18_S2_Demograph_Cfront.mp4
   * Analyzing audio file:  E:\PROJECT_DATA\DEMENTIA_2017\Database\P18\P18_S2_Demograph.wav
      Output exists: E:\PROJECT_DATA\DEME...\P18_S2_Demograph_features.csv
   * Analyzing video:  E:\PROJECT_DATA\DEMENTIA_2017\Database\P18\P18_S2_Demograph_Cfront.mp4
      Running, output generated in:  E:\PROJECT_DATA\DEMENTIA_2017\Database_ANALYSIS\P21_29JAN2019\openface_output
       Execution time: 48.1 min
- 2/15: P18_S2_IAPS_HAPPY_Cfront.mp4
   * Analyzing audio file:  E:\PROJECT_DATA\DEMENTIA_2017\Database\P18\P18_S2_IAPS_HAPPY.wav
      Output exists: E:\PROJECT_DATA\DEME...P18_S2_IAPS_HAPPY_features.csv
   * Analyzing video:  E:\PROJECT_DATA\DEMENTIA_2017\Database\P18\P18_S2_IAPS_HAPPY_Cfront.mp4
      Output exists: E:\PROJECT_DATA\DEME...t\P18_S2_IAPS_HAPPY_Cfront.csv
- 3/15: P18_S2_IAPS_SAD_Cfront.mp4
   * Analyzing audio file:  E:\PROJECT_DATA\DEMENTIA_2017\Database\P18\P18_

## In this section I generate the detections from the time series, both for the video and audio.

In [16]:

for i, (v, a) in enumerate(file_names):
    
    #####################
    # DIRECTORIES
    #####################
    video_output_filename_no_ext = v.split(".")[0]
    openface_csv_file = video_output_filename_no_ext+".csv"
    openface_csv_path = os.path.join(output_openface_dir, openface_csv_file)
    openface_avi_path = os.path.join(output_openface_dir, video_output_filename_no_ext+".avi")
    elan_output_path = os.path.join(output_merged_dir, video_output_filename_no_ext+".eaf")
    detections_merged_output_path = os.path.join(output_merged_dir, video_output_filename_no_ext+"_detections_merged.csv")
    detection_settings_output_path = os.path.join(output_merged_dir, video_output_filename_no_ext+"_detection_settings.csv")

    audio_output_filename_no_ext = a.split(".")[0]
    audio_csv_file = audio_output_filename_no_ext+"_features.csv"
    audio_csv_path = os.path.join(output_audio_dir , audio_csv_file)
    
    print("- "+str(i+1)+"/"+str(len(file_names))+": "+openface_csv_file+", "+audio_csv_file)
    print("     files found: ", os.path.isfile(openface_csv_path) and os.path.isfile(audio_csv_path))
    
    
    #####################
    # OPENFACE
    #####################
    openface_time_series = exploface.get_feature_time_series(openface_csv_path)
    #columns = [c for c in openface_time_series.columns if c in ['confidence', 'success'] or ("AU" in c)]#"_r" in c and
    
    columns = {\
        "name":[],\
        "intensity_threshold":[],\
        "smooth_time_threshold":[],\
        "time_threshold":[],\
        "method":[],\
        "inverse_threshold":[],\
    }
    
    # Default thresholds
    method="threshold"
    inverse_threshold=False
    intensity_threshold=0.8
    time_threshold=0.1
    smooth_time_threshold = 0.1
    uncertainty_threshold=0.9
    
    # Making column name and threshold settings dataframe
    for i, c in enumerate(openface_time_series.columns):
        if c == 'confidence':
            columns["name"].append( c )
            columns["intensity_threshold"].append(intensity_threshold)
            columns["smooth_time_threshold"].append(smooth_time_threshold)
            columns["time_threshold"].append(time_threshold)
            columns["method"].append(method)
            columns["inverse_threshold"].append(True)
        if "AU" in c and "_r" in c:
            columns["name"].append( c )
            columns["intensity_threshold"].append(intensity_threshold)
            columns["smooth_time_threshold"].append(smooth_time_threshold)
            columns["time_threshold"].append(time_threshold)
            columns["method"].append(method)
            columns["inverse_threshold"].append(inverse_threshold)
    
    columns = pd.DataFrame(columns)

    # Running exploface
    openface_detections = exploface.get_detections(openface_time_series, only_columns=columns)
    
    #####################
    # AUDIO
    #####################
    audio_time_series = pd.read_csv(audio_csv_path)
    audio_detections = explibrosa.get_detections(audio_time_series)
       
    
    #####################
    # OUTPUTTING
    #####################
    # Combining openface and the audio
    detections = pd.concat([openface_detections, audio_detections])
    
    elanwriter.write_elan_file(detections, 
                               video_path=openface_avi_path, 
                               output_path=elan_output_path,
                               feature_col_name = "feature")
    
    # Writing out the settings
    columns.to_csv(detection_settings_output_path)
    # And the merged dections
    detections.to_csv(detections_merged_output_path)
    

    


- 1/15: P18_S2_Demograph_Cfront.csv, P18_S2_Demograph_features.csv
     files found:  True
- 2/15: P18_S2_IAPS_HAPPY_Cfront.csv, P18_S2_IAPS_HAPPY_features.csv
     files found:  True
- 3/15: P18_S2_IAPS_SAD_Cfront.csv, P18_S2_IAPS_SAD_features.csv
     files found:  True
- 4/15: P18_S2_LSB_HM1_Cfront.csv, P18_S2_LSB_HM1_features.csv
     files found:  True
- 5/15: P18_S2_LSB_SM1_Cfront.csv, P18_S2_LSB_SM1_features.csv
     files found:  True
- 6/15: P18_S2_LSB_SM2_Cfront.csv, P18_S2_LSB_SM2_features.csv
     files found:  True
- 7/15: P18_S2_Neutral_2_Cfront.csv, P18_S2_Neutral_2_features.csv
     files found:  True
- 8/15: P21_S2_IAPS_HAPPY_C1.csv, P21_S2_IAPS_HAPPY_features.csv
     files found:  True
- 9/15: P21_S2_IAPS_SAD_C1.csv, P21_S2_IAPS_SAD_features.csv
     files found:  True
- 10/15: P21_S2_LSB_HM1_C1.csv, P21_S2_LSB_HM1_features.csv
     files found:  True
- 11/15: P21_S2_LSB_HM2_C1.csv, P21_S2_LSB_HM2_features.csv
     files found:  True
- 12/15: P21_S2_LSB_HM3_C1.csv, P